# Understanding Embedding Layers in PyTorch

This notebook explains the embedding process in PyTorch using an example with a large vocabulary size and embedding dimension.

## What is an Embedding Layer?

An embedding layer maps discrete input tokens (like words) into continuous vectors. It is used to represent words in a dense vector space where each word has its own unique vector representation.

### Why Use Embeddings?
- **Dimensionality Reduction:** Instead of one-hot encoding with a large dimension, embeddings provide a lower-dimensional representation.
- **Semantic Meaning:** Vectors can capture semantic relationships between words.

## Step-by-Step Explanation

### Step 1: Define the Vocabulary
For simplicity, we use a small subset of words to illustrate the process. In practice, the vocabulary can be much larger.

In [70]:
import torch
import torch.nn as nn

# Define a small vocabulary for illustration
vocab = {'<pad>': 0, '<unk>': 1, 'the': 2, 'to': 3, 'and': 4, 'of': 5, 'a': 6, 'in': 7, 'that': 8, 'is': 9, 'cat': 10, 'hat': 11}
vocab_size = len(vocab)


## Step 2: Initialize the Embedding Layer
The embedding layer is initialized with a random matrix of size `(vocab_size, embedding_dim)`.

In [71]:
# Define the embedding dimension
embedding_dim = 512

# Initialize the embedding layer
embedding_layer = nn.Embedding(vocab_size, embedding_dim)

## Step 3: Embedding Matrix Initialization
The embedding matrix \( E \) has dimensions `(vocab_size, embedding_dim)`. Each row represents a word vector in the embedding space.

### Math Behind Embedding Initialization
For each word in the vocabulary, a unique vector of size `embedding_dim` is initialized. This matrix is updated during training to capture the relationships between words.

Let's initialize and inspect the embedding matrix.

In [72]:
# Inspect the size of the embedding matrix
embedding_matrix = embedding_layer.weight.data
embedding_matrix.size()

torch.Size([12, 512])

## Step 4: Input Sequence to Embeddings
Given an input sequence of token indices, the embedding layer retrieves the corresponding vectors from the embedding matrix.

In [73]:
# Example input sequence: "the cat in the hat"
input_indices = torch.tensor([2, 10, 7, 2, 11], dtype=torch.long)  # Example indices based on the vocab

# Get the embeddings for the input sequence
embeddings = embedding_layer(input_indices)

## Step 5: Inspect the Embeddings
Each token index in the input sequence is mapped to its corresponding 512-dimensional vector.

In [74]:
embeddings

tensor([[-0.7316,  0.7371, -1.0426,  ..., -0.1416,  0.6958,  0.7521],
        [-1.2539, -0.7819,  0.6867,  ..., -0.2595,  1.5378,  0.9113],
        [ 1.0973,  1.1544,  0.2164,  ...,  1.4280,  1.2353, -1.2412],
        [-0.7316,  0.7371, -1.0426,  ..., -0.1416,  0.6958,  0.7521],
        [-0.2964,  1.1866, -1.6716,  ...,  0.2394, -1.0284,  0.8791]],
       grad_fn=<EmbeddingBackward0>)

## Explanation of the Embeddings
- Each row in the output corresponds to a token in the input sequence.
- The values in each row are the learned weights representing the token in the 512-dimensional space.
- These embeddings capture the semantic meaning and relationships between words.

For example, words that are similar or often appear in similar contexts will have similar vector representations.

## Recap
1. **Vocabulary Size:** The number of unique tokens in the vocabulary (e.g., 10,000).
2. **Embedding Dimension:** The size of the vector representing each token (e.g., 512).
3. **Embedding Matrix:** A matrix of size `(vocab_size, embedding_dim)` initialized randomly.
4. **Token Indices:** Each word in the input sequence is converted to its corresponding index.
5. **Embedding Lookup:** The embedding layer retrieves the vectors for the input token indices, resulting in a sequence of vectors.

This process allows the model to learn and represent words in a continuous vector space, capturing their semantic relationships and improving the model's performance in tasks like text generation.

## Embeddings Passed to Positional Encoder

In a Transformer architecture, after converting tokens to embeddings, the embeddings are passed through the positional encoder. This step adds positional information to the embeddings, enabling the model to understand the order of the tokens.

## More in-depth look at Embeddings:

In [75]:
import torch
import torch.nn as nn

# Define the input text
text = """
This script implements a decoder-only Transformer model for text generation, similar to the architecture used in GPT (Generative Pre-trained Transformer). The model is trained on a text dataset, specifically the book "Pride and Prejudice," to learn to generate text in the style of the book.
The script trains the model on the text of "Pride and Prejudice" to generate text in a similar style. After training, the model can be used to generate text by predicting the next token in a sequence based on the previous tokens.
"""

# Tokenize the text and create the vocabulary
def create_vocab(text):
    words = text.split()
    print(f"Words: {words}")
    unique_words = set(words) # removes duplicates
    print(f"Unique Words: {words}")
    vocab = {word: i + 4 for i, word in enumerate(unique_words)} # create dict - map each word to a index
    print(f"Vocab: {vocab}")

    # Add special tokens to the vocabulary with fixed indices
    vocab['<pad>'] = 0
    vocab['<unk>'] = 1
    vocab['<sos>'] = 2
    vocab['<eos>'] = 3
    return vocab

vocab = create_vocab(text)
print("Vocabulary size:", len(vocab))
print("Sample vocabulary:", {k: vocab[k] for k in list(vocab)[:10]})

Words: ['This', 'script', 'implements', 'a', 'decoder-only', 'Transformer', 'model', 'for', 'text', 'generation,', 'similar', 'to', 'the', 'architecture', 'used', 'in', 'GPT', '(Generative', 'Pre-trained', 'Transformer).', 'The', 'model', 'is', 'trained', 'on', 'a', 'text', 'dataset,', 'specifically', 'the', 'book', '"Pride', 'and', 'Prejudice,"', 'to', 'learn', 'to', 'generate', 'text', 'in', 'the', 'style', 'of', 'the', 'book.', 'The', 'script', 'trains', 'the', 'model', 'on', 'the', 'text', 'of', '"Pride', 'and', 'Prejudice"', 'to', 'generate', 'text', 'in', 'a', 'similar', 'style.', 'After', 'training,', 'the', 'model', 'can', 'be', 'used', 'to', 'generate', 'text', 'by', 'predicting', 'the', 'next', 'token', 'in', 'a', 'sequence', 'based', 'on', 'the', 'previous', 'tokens.']
Unique Words: ['This', 'script', 'implements', 'a', 'decoder-only', 'Transformer', 'model', 'for', 'text', 'generation,', 'similar', 'to', 'the', 'architecture', 'used', 'in', 'GPT', '(Generative', 'Pre-traine

In [76]:
# Define the embedding layer
vocab_size = len(vocab)  # Example vocabulary size
print(f"Vocab Size: {vocab_size}")

embedding_dim = 10  # Embedding dimension
embedding_layer = nn.Embedding(vocab_size, embedding_dim)
print(f"Embedding Matrix Share: {embedding_layer}")

# Extract the embedding matrix
embedding_matrix = embedding_layer.weight.data

print("Embedding Matrix:")
print(embedding_matrix)


Vocab Size: 54
Embedding Matrix Share: Embedding(54, 10)
Embedding Matrix:
tensor([[ 7.3596e-01,  1.2562e+00, -1.3335e+00,  2.5319e-01, -4.4919e-01,
          1.7723e+00, -1.5894e+00,  3.8627e-01,  2.4122e-01,  1.6544e+00],
        [-1.0111e+00,  4.1950e-01, -4.4773e-01, -1.7319e-01, -5.7057e-01,
          1.8018e-01, -1.1488e+00, -1.8112e+00,  3.3982e-01, -5.2644e-02],
        [ 1.7883e+00, -1.3003e+00,  1.4804e-01, -7.5168e-01,  4.7431e-01,
         -6.3050e-01, -9.9474e-01, -5.7372e-02,  1.9889e-01, -3.6535e-01],
        [ 1.6899e+00, -1.2200e+00, -7.5872e-01, -5.3970e-01, -3.8359e-01,
          1.7771e+00,  4.6346e-01, -6.5572e-01,  1.2301e+00,  1.0581e+00],
        [-1.9759e-02, -1.5920e-02, -1.3658e-01, -1.5800e+00, -1.7770e+00,
          3.2761e-02, -2.2128e-01,  8.9837e-01, -1.7743e+00,  1.1366e-01],
        [-9.7560e-01,  5.2428e-01, -1.0213e+00,  2.6234e-01,  1.0089e-01,
          8.1758e-01, -1.4203e+00, -8.4550e-02, -1.1753e+00,  1.9893e-01],
        [ 7.4582e-02, -1.0402e+

An embedding matrix of shape (54, 10) is created. Each row in this matrix corresponds to a word vector of size 10.  We can also print out the full Embedding Matrix as shown above.

### Extract the First Sentence and Convert to Indices

In [77]:
first_sentence = "This script implements a decoder-only Transformer model for text generation"
first_sentence_indices = [vocab.get(word, vocab['<unk>']) for word in first_sentence.split()]
print(f"Sentence indicies: {first_sentence_indices}")

# Turn it into a tensor for processing
input_indices = torch.tensor(first_sentence_indices, dtype=torch.long)
print(f"Input indices: {input_indices}")


Sentence indicies: [21, 6, 15, 5, 8, 53, 50, 34, 24, 1]
Input indices: tensor([21,  6, 15,  5,  8, 53, 50, 34, 24,  1])


### Get the embeddings for the input sequence

In [78]:
embeddings = embedding_layer(input_indices)

Retrieve the corresponding rows from the embedding matrix  E  for each index.

So, If first_sentence_indices = [21, 6, 15, 8, 53, 50, 34, 24, 1], the corresponding embeddings would be something like this:

In [79]:
embeddings_example = [
 [0.5, 0.6, ..., 0.8],  # Embedding for 'This' (index 21)
 [0.3, 0.4, ..., 0.7],  # Embedding for 'script' (index 6)
 [0.7, 0.1, ..., 0.9],  # Embedding for 'implements' (index 15)
 [0.4, 0.5, ..., 0.6],  # Embedding for 'decoder-only' (index 8)
 [0.2, 0.3, ..., 0.4],  # Embedding for 'Transformer' (index 53)
 [0.4, 0.5, ..., 0.6],  # Embedding for 'model' (index 50)
 [0.1, 0.2, ..., 0.3],  # Embedding for 'for' (index 34)
 [0.3, 0.4, ..., 0.7],  # Embedding for 'text' (index 24)
 [0.3, 0.4, ..., 0.7],  # Embedding for 'generation' (<unk>, index 1)
]

However, let's get the actualy Embeddings from the first_sentence_indices:

In [80]:
print("Embeddings shape: ", embeddings.shape)

print(f"\nfirst_sentence_indices Embeddings: \n\n{embeddings}")

Embeddings shape:  torch.Size([10, 10])

first_sentence_indices Embeddings: 

tensor([[ 0.0171,  0.0654,  0.4616, -0.0804,  0.7193, -1.7430,  0.7120, -0.5802,
          2.7427, -0.7156],
        [ 0.0746, -1.0402,  1.6724,  0.2307,  0.0400,  1.0399,  0.3740, -0.5915,
          1.3554, -2.3176],
        [-1.2554,  0.6687,  0.1003, -0.9104,  1.5644,  1.6488, -0.1967, -1.6907,
         -0.4079,  0.2899],
        [-0.9756,  0.5243, -1.0213,  0.2623,  0.1009,  0.8176, -1.4203, -0.0846,
         -1.1753,  0.1989],
        [ 0.3587, -0.0741, -1.8580,  1.1830, -0.3402, -0.9538,  1.9216, -0.3916,
          0.2070,  0.5739],
        [-0.5651,  0.9522, -0.1525, -0.8550, -0.5317, -1.0827, -1.6945, -0.7518,
         -0.2396, -0.1583],
        [ 0.9786, -2.0795, -0.2498,  0.8052, -0.6686, -0.3186, -0.5592,  0.8016,
          0.7862, -1.9430],
        [-1.0120,  1.3493,  0.9736,  0.2722, -1.9441, -0.1646, -0.9899,  0.8329,
          0.2965,  0.0083],
        [ 0.6008, -0.7061,  2.3409, -0.8130, -0.01